<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/ChatGPT_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**LangChain APP**

https://qiita.com/wwwcojp/items/c7f43c5f964b8db8a890

https://note.com/npaka/n/n6b7a07e492f1

In [1]:
# パッケージのインストール
!pip install langchain --q
!pip install openai --q
!pip install google-search-results --q
!pip install --upgrade deepl --q
import openai
import deepl
import textwrap
import pandas as pd

# パッケージのインポート
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

from google.colab import drive
drive.mount("/content/drive")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 KB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
Mounted at /content/drive


In [2]:
with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
# #ngrok_aceess_token = key[5]
# openai.api_key = key[3]
# deepl_auth_key = key[1]
# serp_api_key = key[7]

import os
os.environ["OPENAI_API_KEY"] = key[3]
os.environ["SERPAPI_API_KEY"] = key[7]
os.environ["GOOGLE_CSE_ID"] = key[9]
os.environ["GOOGLE_API_KEY"] = key[11]

In [ ]:
# LLMの準備準備
llm = OpenAI(temperature=0)

# ツールの準備
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# エージェントの準備
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True
)

In [ ]:
# エージェントの実行
agent.run("おでこにヒアルロン酸を注入する際の合併症は？")



> Entering new AgentExecutor chain...
 I need to find out what the complications of injecting hyaluronic acid into the forehead are.
Action: Search
Action Input: Hyaluronic acid forehead injection complications
Observation: Fortunately, over 90% of adverse events from the use of HA fillers are mild and transient. These include injection site redness, swelling, or bruising. However, disastrous outcomes can occur, including necrosis, vision loss, and cerebrovascular accidents.
Thought: I now know the potential complications of injecting hyaluronic acid into the forehead.
Final Answer: The potential complications of injecting hyaluronic acid into the forehead include injection site redness, swelling, bruising, necrosis, vision loss, and cerebrovascular accidents.

> Finished chain.


'The potential complications of injecting hyaluronic acid into the forehead include injection site redness, swelling, bruising, necrosis, vision loss, and cerebrovascular accidents.'

#**例を見ながら勉強**

## 基本的な入出力

In [ ]:
from langchain.llms import OpenAI

# モデル名を指定しない場合、「text-davinci-003」が利用されます
llm = OpenAI(temperature=0.9)
text = "国立大学法人東京工業大学と国立大学法人東京医科歯科大学が合併しました。ついクスっと笑ってしまう新名称を考えてください。"
print(llm(prompt=text))



「東京テクノロジー医科歯科大学」


## PromptTemplate

ユーザー入力や複数出力に対応

In [ ]:
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["target"],
    template="技術記事のタイトルをよしなに生成してくれるAIの{target}を考えてください。",
)
print(llm(prompt=prompt.format(target="セールスポイント")))



・最新のテクノロジー分野で有効な技術に関する記事タイトルを自動生成するAIを提供します。

・最新の研究成果を内部で解析して、記事タイトルに適したワードを抽出し、効率的な記事タイトルを提供します。

・記事タイトルの作成にお金や時間を節約します。

・テクノロジーを専門とする専門家がいなくても、より良質な記事タイトルをつくることができます。

・新しい技


## Chains

指定した順番で作業を行う

・SimpleSequentialChain : 入出力を1つずつ持つチェーンを繋げるシンプルなチェーン

・SequentialChain : 入出力を複数持つチェーンを繋げる一般的なチェーン

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# プロンプトテンプレートの準備
prompt = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る新会社名の案を1つ出してください",
)

# LLMChainの準備
chain = LLMChain(llm=llm, prompt=prompt)

# LLMChainの実行
print(chain.run("家庭用ロボット"))



ジョイポット・ファミリー


### SimpleSequentialChainの例2つ

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=.7)
template = """あなたは新商品の開発を担当しています。新商品のターゲットが与えられたら、ターゲットに売れる新商品のアイデアを出してください。

ターゲット: {target}
担当者: 以下が新商品のアイデアです。"""
prompt_template = PromptTemplate(input_variables=["target"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

llm = OpenAI(temperature=.7)
template = """あなたは経営者です。新商品のアイデアが与えられたら、経営者の観点から批判的にレビューをしてください。

アイデア:
{idea}
経営者：
"""
prompt_template = PromptTemplate(input_variables=["idea"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)
review = overall_chain.run("通勤に1時間以上かかっている20代の男性")



> Entering new SimpleSequentialChain chain...


1.高性能のノートPC: 20代の男性にとって、ノートPCは非常に便利なツールです。ノートPCは、高性能なパフォーマンスを提供しながら、コンパクトで軽量なので、通勤中の仕事や勉強などに最適です。

2. 無線LAN・モバイルルーター: 通勤中の電車内でもインターネット接続ができる無線LAN・モバイルルーターは、20代の男性にとって便利な商品です。インターネット接続ができるので、電車内でも仕事や勉強を続

1.高性能のノートPC: 確かに、20代の男性にとって高性能のノートPCは非常に便利なツールです。しかし、値段が高くなりすぎると、市場から受け入れられない可能性があります。また、他のノートPCとの差別化を図るため、価値を高めるための新しい機能を提供する必要があるでしょう。

2.無線LAN・モバイルルーター: 確かに、20代の男性にとって便利な商品ですが、他の製品との差別化を行う

> Finished chain.


In [ ]:
# https://note.com/npaka/n/n61ad59380a43

# LLMの定義
llm = OpenAI(temperature=.7)


# 1つ目のチェーンの準備 : 劇のタイトルからあらすじを生成

# テンプレートの準備
template = """あなたは劇作家です。劇のタイトルが与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
あらすじ:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["title"], 
    template=template
)

# LLMChainの準備
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)



# 2つ目のチェーン : 劇のあらすじからレビューを生成

# LLMの準備準備
llm = OpenAI(temperature=.7)

# テンプレートの準備
template = """あなたは演劇評論家です。 劇のあらすじが与えられた場合、そのあらすじのレビューを書くのがあなたの仕事です。

あらすじ:
{synopsis}
レビュー:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
review_chain = LLMChain(llm=llm, prompt=prompt_template)




from langchain.chains import SimpleSequentialChain

# SimpleSequentialChainで2つのチェーンを繋ぐ
overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], 
    verbose=True
)

print(overall_chain("朝の通勤でほっこりしたこと"))

### SequentialChainの例（こちらの方が多分使いやすい）

In [34]:
# 1つ目のチェーン : 劇のタイトルと時代からあらすじを生成

# LLMの準備
llm = OpenAI(temperature=.7)

# テンプレートの準備
# テンプレートの準備
template = """あなたは劇作家です。劇のタイトルと時代が与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
時代:{era}
あらすじ:"""

# プロンプトテンプレートの生成
prompt_template = PromptTemplate(
    input_variables=["title", 'era'], 
    template=template
)

# LLMChainの準備
synopsis_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    output_key="synopsis"
)




# 2つ目のチェーン : 劇のあらすじからレビューを生成

# LLMの準備準備
llm = OpenAI(temperature=.7)

# テンプレートの準備
template = """あなたは演劇評論家です。 劇のあらすじが与えられた場合、そのあらすじのレビューを書くのがあなたの仕事です。

あらすじ:
{synopsis}
レビュー:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
review_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="review"
)



from langchain.chains import SequentialChain

# SequentialChainで2つのチェーンを繋ぐ
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    output_variables=["synopsis", "review"],
    verbose=True
)

In [35]:
# SequentialChainの実行
review = overall_chain({"title":"通勤時にほっこりしたこと", "era": "戦国時代"})



> Entering new SequentialChain chain...

> Finished chain.


In [38]:
print(f"あらすじ：{review['synopsis']}")
print("")
print(f"評論：{review['review']}")

あらすじ：

戦国時代の朝、ある農家の娘と彼女の叔父が、彼女を大都市に連れて行く旅をしていました。彼女は驚いて、そこで何が起こるのか想像できませんでした。着いた先は、大きな城が立ち並ぶ大都市でした。そして娘は彼女の叔父と同じように、農場から大都市への旅を毎朝、通勤する農民の姿を見ていました。農民たちは行動することで、自分の夢を叶えようとしていました。そして、

評論：

この作品は、戦国時代の農家の娘が、叔父と一緒に大都市への旅をし、そこで何が起きるのかを想像できない状況にあるという物語です。この作品の主人公は、農民たちが毎朝行動を起こして、自分の夢を叶えようとしている姿を見て、勇気を得ることができます。作品全体は、戦国時代の大都市の生活を熱望する農民たちの姿を描写しています。この作品は、勇気を持って夢を


## Agents

Chainをどのような順番で動かすかを決定する

・zero-shot-react-description：  Toolのdescription（説明文）をもとに、どのToolを選択するか決定する

・react-docstore:  ドキュメントを検索するSearchと、検索結果から用語を探すLookupという2つのToolを用意

・self-ask-with-search:  Google検索等を用いて、自問自答・検索を繰り返しながら答えを導く





In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("日本で一番高い山の高さは何メートルですか?その高さの5乗根を求めてください。")



> Entering new AgentExecutor chain...
 I need to find the height of the highest mountain in Japan and then calculate its fifth root.
Action: Search
Action Input: "highest mountain in Japan"
Observation: Mount Fuji
Thought: I need to find the height of Mount Fuji
Action: Search
Action Input: "height of Mount Fuji"
Observation: 12,388′
Thought: I need to convert this to meters
Action: Calculator
Action Input: 12,388 * 0.3048

> Entering new LLMMathChain chain...
12,388 * 0.3048
Answer: 3760.544
> Finished chain.

Observation: Answer: 3760.544
Thought: I need to calculate the fifth root of 3760.544
Action: Calculator
Action Input: 3760.544^(1/5)

> Entering new LLMMathChain chain...
3760.544^(1/5)
```python
print(3760.544**(1/5))
```

Answer: 5.188599325914775

> Finished chain.

Observation: Answer: 5.188599325914775

Thought: I now know the final answer
Final Answer: The height of the highest mountain in Japan is 3760.544 meters and its fifth root is 5.188599325914775.

> Finished cha

'The height of the highest mountain in Japan is 3760.544 meters and its fifth root is 5.188599325914775.'

## Memory

前までの会話を覚えておく

・  ConversationBufferMemory  前段までの入力・出力をそのまま記憶する

・  ConversationalBufferWindowMemory  k段前までの入力・出力をそのまま記憶する

・  ConversationSummaryMemory  前段までの入力・出力をLLMでサマライズして記憶する

#**Google custom searchを併用する**

https://note.com/npaka/n/nd9a4a26a8932

In [3]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, load_tools
from langchain import OpenAI, SerpAPIWrapper, LLMChain

# ツールの準備
tools = load_tools(["google-search"], llm=OpenAI())

In [6]:
#テンプレートの準備
prefix = """次の質問にできる限り答えてください。次のツールにアクセスできます:"""
suffix = """始めましょう! 最終的な答えを出すときは、一人称は"ぼく"、語尾には"なのだ"を使用してください

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "agent_scratchpad"]
)

# エージェントの準備
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.run("天才バカボンのパパが卒業した大学は？")





> Entering new AgentExecutor chain...
Thought: パパの卒業大学を探す
Action: Google Search
Action Input: 天才バカボン パパ 卒業大学
Observation: Jul 31, 2018 ... バカボンのパパが「バカ田大学」を卒業（哲学科の首席！）したことは有名な話ですが、その在学中には「黒百合女学院」の“マドンナ”（のちのママ）に恋 ... バカボンのパパ（バカボンパパ）は、赤塚不二夫のギャグ漫画・『天才バカボン』、ならびにそれを ... 最終学歴, バカ田大学 ... 天才バカボン』を第5作と記す。 天才バカボン キャラ紹介. パパバカ田大学を主席で卒業するも、定職に就かず「これでいいのだ！ ... 時に、パパを懸命に怒っている姿さえも愛らしい。 バカ田大学とは赤塚不二夫のギャグ漫画「天才バカボン」に登場する大学。バカボンのパパを輩出したことで有名。 概要 バカ田大学とは赤塚不二夫のギャグ漫画「天才 ... 天才バカボンのパパは、熊本県に実在する菊地市立七城中学校から東京都のバカ田高校を経て、バカ田大学を主席で卒業。かなり多くの職を転々としていますが、店舗を全焼 ... バカ田大学社会学部哲学科出身です。東京の西北早稲田の隣です。生徒の中にはアホウドリもいます。 姉妹校には「アッホー大学」、「バーカード大学」があります。 Sep 11, 2008 ... 天才バカボンの大神秘―バカボンのパパの知能指数は12500なのだ!?作者: バカ田大学後援会出版社/メーカー: ベストセラーズ発売日: 1993/06メディア: ... 名称は早稲田大学のパロディであり、校歌の歌詞もモデルの「都の西北早稲田の森に～」をもじり、「都の東北早稲田の隣～」となっている。バカボンのパパが首席で卒業した ... バカボンのパパは、熊本県金地郡七城町立七城中学校、東京都のバカ田高校を経てバカ田大学を首席で卒業。学級委員も務めています。/ バカボンのパパ（バカボンパパ）は、赤塚不二夫のギャグ漫画・『天才 ... バカ田大学の卒業式の日、「東洋工業（現・マツダ）に入社してマツダ・キャロルを作るのだ」 ...
Thought: パパが卒業した大学はバカ田大学なのだ
Final Answer: パパはバカ田大学

'パパはバカ田大学を首席で卒業したなのだ。'

## 危ないアドレスかどうかを判断する

※これはColabでは動きません...

https://zenn.dev/tatsui/articles/c4b4f796a85395

In [8]:
import os
import sys
import click

from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI, SerpAPIWrapper, LLMChain, LLMMathChain, LLMBashChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, load_tools, initialize_agent

def generate_prompt(identifier:str,input_value:str):
    if identifier in ["url", "domain"]:
        prompt = f"""
以下の手順を使ってURL({input_value})の調査を行い、悪意の有無について「日本語」で出力してください。
* digコマンドを使って入力されたドメイン名からIPアドレスを取得します。
* whoisコマンドを使ってWhois情報を取得します。
* IPアドレスをgoogleで検索する

判定基準
* IP Abuse Reports
"""

    if identifier == "ip":
        prompt = f"""
以下の手順を使ってIPアドレス({input_value})の調査を行い、悪意の有無について「日本語」で出力してください。
* whoisコマンドを使ってWhois情報を取得します。
* IPアドレスをgoogleで検索する

判定基準
* 悪意のあるIPアドレスとして報告されている
* IP Abuse Reports
"""

    if identifier == "hash":
       prompt = f"""
以下の手順を使ってHash({input_value})の調査を行い、悪意の有無について「日本語」で出力してください。
* ハッシュ値をGoogle検索する

判定基準
* 悪意のあるファイルとして報告されている
"""
    return prompt

@click.command()
@click.option("--url", "identifier", flag_value="url", help="Investigate based on URL.")
@click.option("--hash", "identifier", flag_value="hash", help="Investigate based on file hash.")
@click.option("--domain", "identifier", flag_value="domain", help="Investigate based on domain name.")
@click.option("--ip", "identifier", flag_value="ip", help="Investigate based on IP address.")
@click.argument("input_value")
def main(identifier: str, input_value: str):
    # ツールの準備
    llm = OpenAI(temperature=0)
    tools = load_tools(["terminal", "google-search"], llm=llm)
    llm_math_chain = LLMMathChain(llm=llm, verbose=True)
    llm_bash_chain = LLMBashChain(llm=llm, verbose=True)
    tools.append(
        Tool(
            name="Calculator",
            func=llm_math_chain.run,
            description="useful for when you need to answer questions about math",
        )
    )

    memory = ConversationBufferMemory(memory_key="chat_history")
    prompt = generate_prompt(identifier, input_value)
    agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
    agent.run(prompt)

if __name__ == "__main__":
    main()

Usage: ipykernel_launcher.py [OPTIONS] INPUT_VALUE
Try 'ipykernel_launcher.py --help' for help.

Error: no such option: -f


SystemExit: ignored

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#**AI chatbot**

https://qiita.com/hideki/items/b26154ab503fd3394a0b

In [ ]:
!pip install langchain --q
!pip install openai --q
!pip install gradio --q

In [ ]:
import gradio as gr
import os
from langchain import OpenAI, ConversationChain

os.environ['OPENAI_API_KEY'] = key[3]

llm = OpenAI(temperature=0.95)
conversation = ConversationChain(llm=llm, verbose=False)


def chat(message, history):
    history = history or []
    response = conversation.predict(input=message)
    history.append((message, response))

    return history, history


chatbot = gr.Chatbot().style(color_map=('green', 'pink'))
demo = gr.Interface(
    chat,
    ['text', 'state'],
    [chatbot, 'state'],
    allow_flagging='never',
)
if __name__ == '__main__':
    demo.launch()

### コードエンジニアリングをしてみる

In [13]:
import gradio as gr
import os
from langchain import OpenAI, ConversationChain
from langchain.prompts import PromptTemplate

os.environ['OPENAI_API_KEY'] = key[3]

llm = OpenAI(temperature=1)
conversation = ConversationChain(llm=llm, verbose=False)

template = '''I need you to act as a research assistant for a scientific article I am writing. I will provide you with specific questions and information, and I expect you to find relevant and credible sources to support my research. Your responses should include a summary of the main points from the sources, along with the full citation in the desired format (APA, MLA, etc.). Additionally, you should flag any potential issues or limitations in the sources you provide. The title of the article is: Real-time smartphone identification of corneal diseases displayed on computer screen  using embedded deep-learning model: a validation study

{history}
{input}
'''

prompt = PromptTemplate(
    input_variables=['history', 'input'],
    template=template
)

conversation = ConversationChain(
    llm=llm,
    prompt=prompt,
    verbose=True)


def chat(message, history):
    history = history or []
    response = conversation.predict(input=message)
    history.append((message, response))

    return history, history


chatbot = gr.Chatbot().style(color_map=('green', 'pink'))
demo = gr.Interface(
    chat,
    ['text', 'state'],
    [chatbot, 'state'],
    allow_flagging='never',
)
if __name__ == '__main__':
    demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>